In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Новая папка (2)\База 07-08_fixed.xlsx")

In [30]:
df['Дата обновления'] = pd.to_datetime(df['Дата обновления'], errors='coerce')

In [3]:
# Исключаем квартиры от Фонда Реновации
df_filtered = df[df['Девелопер'] != 'Фонд реновации'].copy()

# Добавляем колонку с месяцем (числовой формат — June = 6, July = 7)
df_filtered['месяц'] = df_filtered['Дата обновления'].dt.month

# Считаем количество квартир по проекту и месяцу
project_month_counts = df_filtered.groupby(['Название проекта', 'месяц']).size().unstack(fill_value=0)

# Фильтруем: < 10 квартир в июне и 0 квартир в июле
filtered_projects = project_month_counts[
    (project_month_counts.get(7, 0) < 100000) &  # Июнь
    (project_month_counts.get(8, 0) == 0)    # Июль
]

# Получаем список названий проектов
result = filtered_projects.index.tolist()

In [4]:
result

['Armani/Casa Moscow Residences',
 'Level Нагатинская',
 'Nagatino i-Land',
 'The Patricks',
 'Берег',
 'Высокие Жаворонки',
 'Гармония парк',
 'Донской (Коломна)',
 'Космос',
 'Либерти',
 'Мишино-2',
 'Мой город',
 'Онегин',
 'Первый Нагатинский',
 'Планерный квартал',
 'Подлипки-Город',
 'Пречистенка 8',
 'Ривер хаус',
 'Сабурово Клаб',
 'Серебро']

In [7]:
# Шаг 1 — исключаем квартиры от Фонда Реновации
df_filtered = df[df['Девелопер'] != 'Фонд реновации'].copy()

# Шаг 2 — добавляем колонку с числовым месяцем
df_filtered['месяц'] = df_filtered['Дата обновления'].dt.month

# Шаг 3 — группируем по проекту и месяцу
project_month_counts = df_filtered.groupby(['Название проекта', 'месяц']).size().unstack(fill_value=0)

# Шаг 4 — отбираем проекты: <10 квартир в июне и 0 в июле
filtered_projects = project_month_counts[
    (project_month_counts.get(7, 0) < 100000) & 
    (project_month_counts.get(8, 0) == 0)
]

# Шаг 5 — формируем список строк в нужном формате
result = [
    f"{project} — {count} квартир в Июле"
    for project, count in filtered_projects[7].items()
]

# Вывод
for line in result:
    print(line)

Armani/Casa Moscow Residences — 6 квартир в Июле
Level Нагатинская — 1 квартир в Июле
Nagatino i-Land — 80 квартир в Июле
The Patricks — 1 квартир в Июле
Берег — 189 квартир в Июле
Высокие Жаворонки — 4 квартир в Июле
Гармония парк — 262 квартир в Июле
Донской (Коломна) — 44 квартир в Июле
Космос — 906 квартир в Июле
Либерти — 1 квартир в Июле
Мишино-2 — 56 квартир в Июле
Мой город — 5 квартир в Июле
Онегин — 200 квартир в Июле
Первый Нагатинский — 88 квартир в Июле
Планерный квартал — 270 квартир в Июле
Подлипки-Город — 16 квартир в Июле
Пречистенка 8 — 4 квартир в Июле
Ривер хаус — 124 квартир в Июле
Сабурово Клаб — 24 квартир в Июле
Серебро — 809 квартир в Июле


In [11]:
# Шаг 1 — исключаем квартиры от Фонда Реновации
df_filtered = df[df['Девелопер'] != 'Фонд реновации'].copy()

# Шаг 2 — добавляем числовой месяц
df_filtered['месяц'] = df_filtered['Дата обновления'].dt.month

# Шаг 3 — группировка по проекту и девелоперу + подсчёт квартир по месяцам
project_month_counts = (
    df_filtered
    .groupby(['Название проекта', 'Девелопер', 'месяц'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={7: 'июль', 8: 'август'})
)
project_month_counts = project_month_counts.rename(columns={7: 'июль', 8: 'август'})

# Шаг 4 — фильтрация:
# - минимум 5 квартир в июне
# - июль не ноль
# - изменение >= 80% в любую сторону
filtered = project_month_counts[
    (project_month_counts['июль'] >= 1) &
    (project_month_counts['август'] >= 0) &
    (
        (project_month_counts['август'] >= project_month_counts['июль'] * 2) |
        (project_month_counts['август'] <= project_month_counts['июль'] * 0.2)
    )
]

# Шаг 5 — читаемый вывод
result = [
    f"{project} — Июль: {row['июль']}, Август: {row['август']}"
    for project, row in filtered.iterrows()
]

# Вывод
for line in result:
    print(line)

('Armani/Casa Moscow Residences', 'VOS’HOD') — Июль: 6, Август: 0
('Level Нагатинская', 'Level') — Июль: 1, Август: 0
('Nagatino i-Land', 'Эталон') — Июль: 80, Август: 0
('Riverpark', 'Аеон') — Июль: 386, Август: 0
('The Patricks', 'Третий Рим') — Июль: 1, Август: 0
('Берег', 'СЭСМА') — Июль: 189, Август: 0
('Высокие Жаворонки', 'Гранель') — Июль: 4, Август: 0
('Гармония парк', 'ТКС РИЭЛТИ') — Июль: 262, Август: 0
('Донской (Коломна)', 'РК-Газсетьсервис') — Июль: 44, Август: 0
('Истомкино Парк', 'Тройка плюс') — Июль: 26, Август: 70
('Квартал Ивакино', 'Самолет') — Июль: 37, Август: 7
('Космос', 'СЗ Скала') — Июль: 906, Август: 0
('Либерти', 'Магнум') — Июль: 1, Август: 0
('Михайловский парк', 'ПИК') — Июль: 5, Август: 1
('Мишино-2', 'СЗ Ю-ИНВЕСТ') — Июль: 56, Август: 0
('Мой город', 'Стройспецмонтаж') — Июль: 5, Август: 0
('Ново-Никольское', 'Агрострой') — Июль: 52, Август: 9
('Одинцовские кварталы', 'Стройтек') — Июль: 13, Август: 0
('Онегин', 'ГК Развитие') — Июль: 200, Август: 0
('